In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
import urllib.parse

report_date = '10/31/2024'
# report_date = '12/31/2022'

end = datetime.strptime(report_date, '%m/%d/%Y')
start = date(end.year, end.month, 1)
# start = date(end.year, 1, 1)

import config
base_url = config.church_domain_url + r"/events/reports/243292484#/"

import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(end.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292484#/start_date=10%2F01%2F2024&end_date=10%2F31%2F2024&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-11-28-2024.xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-11-28-2024.xlsx"

# Paste this file into this month's tab in https://docs.google.com/spreadsheets/d/1hAdgMh7_m73L--bR-rUT2dZPMwEJ8s2e/edit#gid=1826328980
paste_file = r"C:\Users\ralph\Downloads\paste_me.csv"
summary_file = r"C:\Users\ralph\Downloads\USDA_summary.csv"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)

# drop the Total row
attendance = attendance[attendance['Breeze ID'].notna()]

people = pd.read_excel(peopleexport)
print('{count} people in input.'.format(count = len(attendance.index)))

186 people in input.


In [3]:
display(attendance.columns)

Index(['Breeze ID', 'First Name', 'Last Name', 'Oct 3, 2024', 'Oct 10, 2024',
       'Oct 17, 2024', 'Oct 24, 2024', 'Oct 31, 2024', 'Person Totals'],
      dtype='object')

In [4]:
# Keep only the columns we use.

attendance = attendance[[
    'Breeze ID', 
    'First Name', 
    'Last Name', 
    'Person Totals'
]]

people = people[[
    'Breeze ID',
    'Date of First Contact',
    'How many people live in your household? (including yourself)',
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
    'City'
]]


In [5]:
# Include household numbers from the people profiles.

report = pd.merge(attendance, people, on = 'Breeze ID', how = "inner")
report.fillna(0, inplace=True)

# print('{count} people in report.'.format(count = len(report.index)))

In [6]:
# Clean up the City column.

def clean(word):
    if word == 0:
        return 'unknown'
    return word.strip().title()
    
report['City'] = report['City'].apply(clean)

In [7]:
# Validate the data 
from pandas.api.types import is_numeric_dtype

# Check for numbers in numeric fields
numeric_fields = [
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
]

for field in numeric_fields:
    if len(report[~report.map(np.isreal)[field]]) > 0:
        print('Invalid numeric data')
        display(field, report[~report.map(np.isreal)[field]])

# fillna() fills with a float, so make it an int.
for column in numeric_fields:
    report[column] = report[column].astype(int)

bad_in_house = report.loc[report['How many people live in your household? (including yourself)'] == 0] 
if len(bad_in_house):
    print("Missing 'Total in house' for")
    display(report.loc[report['How many people live in your household? (including yourself)'] == 0])

bad_totals = report.loc[report['How many people live in your household? (including yourself)'] != 
        report['Total number of people aged 18 or under in household?'] + 
        report['Total number of people between the ages of 19-59 in household?'] + 
        report['Total number of people aged 60 or older in Household?']
    ]
if len(bad_totals) > 0:
    print("Age breakdowns do not add up to total for")
    display(bad_totals)

In [8]:
report.to_csv(paste_file, index=False)

In [9]:
def summarize(report):
    MealsPerWeek = 15
    numberOfSeniors = 25
    max_person_totals = attendance['Person Totals'].max()
    
    SeniorCenter = {
        'Households': numberOfSeniors,
        'Individuals': numberOfSeniors * max_person_totals,
        'Meals': MealsPerWeek * numberOfSeniors * max_person_totals,
        'Over 60': numberOfSeniors * max_person_totals
    }

    def calculate_row(filtered_data, town=None):
        row = {
            'Town': town or 'TOTAL',
            'Households': len(filtered_data),
            'Individuals': (filtered_data['How many people live in your household? (including yourself)'] * filtered_data['Person Totals']).sum(),
            'Meals': MealsPerWeek * (filtered_data['How many people live in your household? (including yourself)'] * filtered_data['Person Totals']).sum(),
            'Over 60': (filtered_data['Total number of people aged 60 or older in Household?'] * filtered_data['Person Totals']).sum(),
            'Children': (filtered_data['Total number of people aged 18 or under in household?'] * filtered_data['Person Totals']).sum()
        }
        if town == 'Newmarket' or town is None:
            for key, value in SeniorCenter.items():
                row[key] += value
        return row

    summary = []

    # Calculate rows for each town
    for town in report['City'].unique():
        filtered = report.loc[report['City'] == town]
        summary.append(calculate_row(filtered, town))

    # Calculate total row
    summary.append(calculate_row(report))

    return summary


In [10]:
# Create the summary for the USDA monthly report.

summary = summarize(report)
pd.DataFrame(summary).to_csv(summary_file, index=False)

# USDA has requested that we report only counts for Newmarket and Newfields. 
# The Pantry has decided that any people not from Newfields will be reported as from Newmarket. 
# To get the full report, comment out this line.
report.loc[report['City'] != 'Newfields', 'City'] = 'Newmarket'
summary = summarize(report)
display(pd.DataFrame(summary))


,Town,Households,Individuals,Meals,Over 60,Children
0,Newmarket,205,1390,20850,385,330
1,Newfields,6,28,420,13,4
2,TOTAL,211,1418,21270,398,334
